# **Aplicação do SVM e Random Forest, com Random Search  e validação cruzada estratificada (Métrica utilizada F-score)**

> Conjunto de dados de pacientes hepáticos indianos
Este conjunto de dados contém 416 registros de pacientes hepáticos e 167 registros de pacientes não hepáticos. O conjunto de dados foi coletado no nordeste de Andhra Pradesh, Índia. O rótulo da classe divide os pacientes em 2 grupos (doente hepático ou não). Este conjunto de dados contém 441 registros de pacientes do sexo masculino e 142 registros de pacientes do sexo feminino.

**Informações dos atributos**
*   V1. Idade do paciente. Qualquer paciente cuja idade ultrapasse 89 anos é listado como tendo idade "90".
*   V2. Sexo do paciente
*   V2. Sexo do paciente
*   V3. Bilirrubina Total
*   V4. Bilirrubina Direta
*   V5. Alkphos Fosfatase Alcalina
*   V6. Sgpt Alanina Aminotransferase
*   V7. Sgot Aspartato Aminotransferase
*   V8. Proteínas Totais
*   V9. Albumina
*   V10. Relação A/G Albumina e Relação Globulina

[Fonte dos dados](https://www.openml.org/search?type=data&sort=runs&id=1480&status=active)


# **Coleta, exploração e pré-processamento dos dados**

In [94]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.io.arff import loadarff 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.metrics import f1_score, make_scorer

In [95]:
data = loadarff('ilpd.arff')
df = pd.DataFrame(data[0])
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
0,65.0,b'Female',0.7,0.1,187.0,16.0,18.0,6.8,3.3,0.90,b'1'
1,62.0,b'Male',10.9,5.5,699.0,64.0,100.0,7.5,3.2,0.74,b'1'
2,62.0,b'Male',7.3,4.1,490.0,60.0,68.0,7.0,3.3,0.89,b'1'
3,58.0,b'Male',1.0,0.4,182.0,14.0,20.0,6.8,3.4,1.00,b'1'
4,72.0,b'Male',3.9,2.0,195.0,27.0,59.0,7.3,2.4,0.40,b'1'


In [96]:
df.describe()

,V1,V3,V4,V5,V6,V7,V8,V9,V10
count,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000
mean,44.746141,3.298799,1.486106,290.576329,80.713551,109.910806,6.483190,3.141852,0.947064
std,16.189833,6.209522,2.808498,242.937989,182.620356,288.918529,1.085451,0.795519,0.318492
min,4.000000,0.400000,0.100000,63.000000,10.000000,10.000000,2.700000,0.900000,0.300000
25%,33.000000,0.800000,0.200000,175.500000,23.000000,25.000000,5.800000,2.600000,0.700000
50%,45.000000,1.000000,0.300000,208.000000,35.000000,42.000000,6.600000,3.100000,0.947064
75%,58.000000,2.600000,1.300000,298.000000,60.500000,87.000000,7.200000,3.800000,1.100000
max,90.000000,75.000000,19.700000,2110.000000,2000.000000,4929.000000,9.600000,5.500000,2.800000


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      583 non-null    float64
 1   V2      583 non-null    object 
 2   V3      583 non-null    float64
 3   V4      583 non-null    float64
 4   V5      583 non-null    float64
 5   V6      583 non-null    float64
 6   V7      583 non-null    float64
 7   V8      583 non-null    float64
 8   V9      583 non-null    float64
 9   V10     583 non-null    float64
 10  Class   583 non-null    object 
dtypes: float64(9), object(2)
memory usage: 50.2+ KB


In [98]:
df.shape

(583, 11)

In [99]:
# Otimizando os dados da feature V2
df['V2'] = df['V2'].astype(str).replace(to_replace=('b',"'"), value='', regex=True)
df['V2'] = df['V2'].map({'Female': 0, 'Male': 1})
df = pd.get_dummies(df, columns=['V2'])

In [100]:
# Otimizando os dados da predictor class
df['Class'] = df['Class'].astype(str).replace(to_replace=('b',"'"), value='', regex=True).astype(np.int64)

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      583 non-null    float64
 1   V3      583 non-null    float64
 2   V4      583 non-null    float64
 3   V5      583 non-null    float64
 4   V6      583 non-null    float64
 5   V7      583 non-null    float64
 6   V8      583 non-null    float64
 7   V9      583 non-null    float64
 8   V10     583 non-null    float64
 9   Class   583 non-null    int64  
 10  V2_0    583 non-null    uint8  
 11  V2_1    583 non-null    uint8  
dtypes: float64(9), int64(1), uint8(2)
memory usage: 46.8 KB


In [102]:
#dividindo a predicton class das features
X = np.array(df.drop('Class', axis=1))
y = np.array(df['Class'])

In [103]:
X.shape

(583, 11)

In [104]:
y.shape

(583,)

# **Croos Validation e Random Search no SVM**

In [105]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits=10)

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#definindo hiperparâmetros
distributions = dict(kernel = ['sigmoid', 'poly','rbf'], C = uniform(loc=1, scale=10))

#instânciando meu classificador
classifier = SVC()

#modelando o grid search com a validação cruzada.
random_SVM = RandomizedSearchCV(classifier, distributions, cv=cv_strat, scoring=f1, random_state=5762, n_iter=5)
random_SVM.fit(X, y)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   estimator=SVC(), n_iter=5,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff38cea9f10>,
                                        'kernel': ['sigmoid', 'poly', 'rbf']},
                   random_state=5762, scoring=make_scorer(f1_score))

In [106]:
print('Melhor resultado f1:', random_SVM.best_score_)
print('\nMelhor configuração de hiperparâmetros:', random_SVM.best_params_)
print( '\nMelhor Configuração encontrada pelo GridSearch: \n', random_SVM.best_estimator_)

Melhor resultado f1: 0.832818081808181

Melhor configuração de hiperparâmetros: {'C': 3.996689355015553, 'kernel': 'poly'}

Melhor Configuração encontrada pelo GridSearch: 
 SVC(C=3.996689355015553, kernel='poly')


# **Croos Validation e Random Search no Random Forest**

In [107]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits=10)

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#definindo hiperparâmetros
distributions1 = dict(n_estimators = randint(10, 1000), bootstrap = [True, False], criterion = ['gini', 'entropy'])

#instânciando meu classificador
classifier1 = RandomForestClassifier(random_state=5762)

#modelando o grid search com a validação cruzada.
random_RF = RandomizedSearchCV(classifier1, distributions1, cv = cv_strat, scoring = f1, random_state=5762, n_iter=5)
random_RF.fit(X, y)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   estimator=RandomForestClassifier(random_state=5762),
                   n_iter=5,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff38cd82210>},
                   random_state=5762, scoring=make_scorer(f1_score))

In [108]:
print('Melhor resultado f1:', random_RF.best_score_)
print('\nMelhor configuração de hiperparâmetros:', random_RF.best_params_)
print( '\nMelhor Configuração encontrada pelo GridSearch: \n', random_RF.best_estimator_)

Melhor resultado f1: 0.8120092932415195

Melhor configuração de hiperparâmetros: {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 488}

Melhor Configuração encontrada pelo GridSearch: 
 RandomForestClassifier(n_estimators=488, random_state=5762)
